# Common modulus attack

If the same message is encrypted with two different RSA public keys with the same n, the message can be decrypted without the private key.


Firstly, create two key pairs sharing the same n. Use the cryptography library to create the second key directly from a known n. The methods private_numbers and public_numbers manipulate data elements such as n, d, or e.


Then, your program, given two versions of the same message that has been encrypted with two RSA public keys (with the same n, but a different e), should be able to decrypt the message.

In [ ]:
import argparse
import sys
from cryptography.hazmat.primitives.asymmetric.rsa import generate_private_key, RSAPublicNumbers, RSAPublicKey, RSAPrivateKey, rsa_recover_prime_factors
import gmpy2


In [ ]:
def bytes_to_int(text: bytes):
    return int.from_bytes(text, 'big')


def int_to_bytes(num):
    num_int = int(num)
    return int.to_bytes(num_int, (num_int.bit_length()+7)//8, "big")


def encrypt(plain: bytes, e: int, n: int, block_size: int | None = None):
    if block_size is None:
        block_size = (n.bit_length()+7)//8

    blocks = [bytes_to_int(plain[i:i+block_size])
              for i in range(0, len(plain), block_size)]

    return b"".join([int_to_bytes(gmpy2.powmod(block, e, n)) for block in blocks])


def decrypt(cipher: bytes, d: int, n: int, block_size: int | None = None):
    if block_size is None:
        block_size = (n.bit_length()+7)//8

    blocks = [bytes_to_int(cipher[i:i+block_size])
              for i in range(0, len(cipher), block_size)]

    return b"".join([int_to_bytes(gmpy2.powmod(block, d, n)) for block in blocks])


In [ ]:


def encrypt_common_modulus(plain: bytes, key1: RSAPublicNumbers, key2: RSAPublicNumbers):
    return encrypt(plain, key1.e, key1.n), encrypt(plain, key2.e, key2.n)


def common_modulus_known_d(cipher: bytes, key1: RSAPublicKey, key2: RSAPrivateKey):
    n = key1.public_numbers().n
    e1, e2 = key1.public_numbers().e, key2.public_key().public_numbers().e
    d2 = key2.private_numbers().d

    p, q = rsa_recover_prime_factors(n, e2, d2)
    phi = (p-1)*(q-1)

    d1 = gmpy2.invert(e1, phi)

    plain = decrypt(cipher, d1, n)
    return plain


def common_modulus_unknown_d(cipher: tuple[bytes, bytes], key1: RSAPublicNumbers, key2: RSAPublicNumbers, block_size: int | None = None):
    n = key1.n

    if block_size is None:
        block_size = (n.bit_length()+7)//8

    e1, e2 = key1.e, key2.e

    gcd, x, y = gmpy2.gcdext(e1, e2)

    if gcd != 1:
        print("Error, GCD(e1,e2) must be 1")
        exit(1)

    blocks = [(bytes_to_int(cipher[0][i:i+block_size]), bytes_to_int(cipher[1][i:i+block_size]))
              for i in range(0, len(cipher[0]), block_size)]

    plain = b''.join([int_to_bytes((gmpy2.powmod(block1, x, n) *
                                    gmpy2.powmod(block2, y, n)) % n)for block1, block2 in blocks])
    return plain



In [ ]:

def main(arguments):
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument(
        "-p", "--plaintext", help="File to be encrypted", type=str)
    parser.add_argument(
        "-s", "--passive", help="The attacker doesn't have a private/public key pair", action=argparse.BooleanOptionalAction, default=False)

    args = parser.parse_args(arguments)

    if args.plaintext:
        plain = args.plaintext.read().encode()
    else:
        plain = b'sample_message'

    if args.passive:
        pub_key1 = generate_private_key(
            0x10001, 2048).public_key().public_numbers()
        pub_key2 = RSAPublicNumbers(7, pub_key1.n)

        encrypted = encrypt_common_modulus(plain, pub_key1, pub_key2)
        print(common_modulus_unknown_d(encrypted, pub_key1, pub_key2).decode())
    else:
        prv_key2 = generate_private_key(0x10001, 2048)
        pub_key2 = prv_key2.public_key()

        n = pub_key2.public_numbers().n
        e2, d2 = pub_key2.public_numbers().e, prv_key2.private_numbers().d

        # OK, you got me. I am using the attack to find a suitable e1.
        # I am simulating what the entity generating the keys would do
        # before handing out the keys
        p, q = rsa_recover_prime_factors(n, e2, d2)
        phi = (p-1)*(q-1)

        for e1 in range(3, 2**2048):
            if gmpy2.gcd(e1, phi) == 1:
                break
        else:
            raise ValueError("e1 couldn't be found")

        pub_key1 = RSAPublicNumbers(
            e1, prv_key2.public_key().public_numbers().n).public_key()

        encrypted = encrypt(plain, pub_key1.public_numbers().e,
                            pub_key1.public_numbers().n)

        plain = common_modulus_known_d(encrypted, pub_key1, prv_key2)

        print(plain.decode())




In [ ]:
sys.argv = [
    "", 
    ]
main(sys.argv[1:])